# Classification

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from skimage.transform import resize

In [2]:
# Get the data
d = pd.read_csv('ClipsPCA30.csv', header = None)
data_source = d.to_numpy()
Labels = data_source[:,-1]

# Remove the smallest classes
index_small_classes = np.where((Labels == 1) | (Labels == 4))[0]
data_source = np.delete(data_source, index_small_classes, axis = 0)

Data = data_source[:,:-1]
Labels = data_source[:,-1]

u = {'GL', 'RG', 'SC', 'Negatives'} # label names
lv = [0, 2, 3, 9] # label values
print(Data.shape)

(8962, 30)


In [3]:
# Get HOG data
dd = pd.read_csv('ClipsHOG.csv', header = None)
DataHOG = dd.to_numpy() # no labels
DataHOG = np.delete(DataHOG, index_small_classes, axis = 0)
print(DataHOG.shape)

(8962, 135)


In [4]:
# Classifiers
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix


#cl_names = ['Linear Discriminant Analysis', '3-nn', 'Decision Tree',
#            'SVM', 'Bagging', 'Random Forest'] # classifiers

cl_names = ['Linear Discriminant Analysis', '3-nn', 'Decision Tree',
            'Bagging', 'Random Forest'] # classifiers

cla = [] # accumlate the classifiers
cla.append(LinearDiscriminantAnalysis())
cla.append(KNeighborsClassifier())
cla.append(DecisionTreeClassifier())
# cla.append(SVC(gamma=0.1, kernel="rbf")) # Not very good here
cla.append(BaggingClassifier())
cla.append(RandomForestClassifier())
    
# cv_results = cross_validate(cla[0],Data,Labels,return_train_score = True, cv = 10)    
# print(np.mean(cv_results['test_score']))
# print(np.mean(cv_results['train_score']))

print('                --Classifier--  PCA30    HOG')
print('--------------------------------------------')

for i in range(len(cl_names)):
    classifier = cla[i]
    y = cross_val_predict(classifier,Data,Labels, cv = 10)    
    yhog = cross_val_predict(classifier,DataHOG,Labels, cv = 10)    
    print('%30s %.4f %.4f' % (cl_names[i],np.mean(y == Labels),np.mean(yhog == Labels)))
    # print(confusion_matrix(Labels,y))
    
    
    

                --Classifier--  PCA30    HOG
--------------------------------------------
  Linear Discriminant Analysis 0.9445 0.8777
                          3-nn 0.9657 0.7897
                 Decision Tree 0.9360 0.7857
                       Bagging 0.9559 0.8378
                 Random Forest 0.9621 0.8662


One of the best options is the LDA. It is imple, fast, reasonably accurate, and can return probabilities. Maybe Random Forest?

In [5]:
import pickle as pk

c1 = cla[0].fit(Data,Labels)
print('LDA resub accuracy = ', np.mean(Labels == c1.predict(Data)))
c2 = cla[-1].fit(Data,Labels)
print('RF resub accuracy = ', np.mean(Labels == c2.predict(Data)))

pk.dump(c1, open("LDA.pkl","wb")) 
pk.dump(c2, open("RF.pkl","wb")) 

# later reload the pickle file
# pca_reload = pk.load(open("pca.pkl",'rb'))
# result_new = pca_reload .transform(X)

LDA resub accuracy =  0.9492300825708547
RF resub accuracy =  1.0
